In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import tifffile
import sys, time, os, csv, warnings
import cProfile, pstats
from jutils import tqdm_alias as tqdm

import psfmatrix, lfimage
import projector, lfdeconv
import special_fftconvolve as special
import jutils as util
import py_light_field as plf

In [ ]:
projectorClass = projector.Projector_allC
inputImage = lfimage.LoadLightFieldTiff('/Users/jonny/Movies/Nils files/Rectified/Left/Cam_Left_40_X1_N19.tif')
hMatrix = psfmatrix.LoadMatrix('PSFmatrix/reducedPSFMatrix_M22.2NA0.5MLPitch125fml3125from-156to156zspacing4Nnum19lambda520n1.33.mat')

In [ ]:
if True:
    # Timing measurements
    
    # Number of timepoints to process
    tileFactor = 1
    # Number of z planes to process
    planesToRun = range(0,1)

    inputImageXN = np.tile(inputImage[np.newaxis,:,:],(tileFactor,1,1))
    # Prime the cache first
    Htf = lfdeconv.BackwardProjectACC(hMatrix, inputImageXN, planes=planesToRun, progress=tqdm, logPrint=False, projector=projectorClass())
    # Now actually measure
    plf.SetStatsFile('timestats.txt', False)
    Htf = lfdeconv.BackwardProjectACC(hMatrix, inputImageXN, planes=planesToRun, progress=tqdm, logPrint=False, projector=projectorClass())
    plf.SetStatsFile('', True)

In [ ]:
rows = []
with open('threads.txt') as f:
    cf = csv.reader(f, delimiter='\t')
    for row in cf:
        rows.append(np.array(row).astype(np.double))
rows = np.array(rows)
t0 = rows[0,2]
dt = rows[:,3]-rows[:,2]
mt = (rows[:,5]-rows[:,4]) + (rows[:,7]-rows[:,6])
col = ['red', 'green', 'blue']
lab = ['FFT', 'mirror', 'convolve']
reportWaits = 10
runTime = 0
mutexTime = 0
numThreads = int(np.max(rows[:,0]))+1
plt.figure(figsize=(14,4))
plt.title('Thread breakdown')
for w in range(3):
    for t in range(numThreads):
        x = []; y = []
        mx = []; my = []
        intermediateTimes = []
        for r in rows:
            if (r[0] == t) and (r[1] == w):
                x.extend([r[2], r[2], r[3], r[3]])
                y.extend([0,1,1,0])
                runTime += r[3]-r[2]
                if (w == 2):
                    if ((r[5]-r[4]) > 5e-6):
                        if (reportWaits):
                            print("Mutex wait: %le"%(r[5]-r[4]))
                            reportWaits = reportWaits - 1
                        mx.extend([r[4], r[4], r[5], r[5]])
                        my.extend([0,1,1,0])
                    mutexTime += r[5]-r[4]
                    if (r[6] != 0) and ((r[7]-r[6]) > 1e-5):
                        if (reportWaits):
                            print("Mutex wait: %le"%(r[7]-r[6]))
                            reportWaits = reportWaits - 1
                        mx.extend([r[6], r[6], r[7], r[7]])
                        my.extend([0,1,1,0])
                    mutexTime += r[7]-r[6]
                if (w == 0):
                    intermediateTimes.extend([r[5], r[7]])

                        
        x = np.array(x) - t0
        y = np.array(y)
        mx = np.array(mx) - t0
        my = np.array(my)
        intermediateTimes = np.array(intermediateTimes) - t0
        if t == 0:
            thisLabel = lab[w]
        else:
            thisLabel = None
        # Show the time spent working on a task
        plt.fill_between(x, t, t+y/2, color=col[w], where=y.astype(np.bool), label=thisLabel)
        # Show the time spent waiting to acquire an accum mutex.
        plt.fill_between(mx, t+0.1, t+0.1+my*0.3, where=my.astype(np.bool), color='gray')
        if (w == 0) and False:
            plt.plot(intermediateTimes, [t+0.25]*len(intermediateTimes), 'x', color='black')
if (reportWaits == 0):
    warnings.warn('Did not report all waits')

# Do a separate plot to examine what fraction of the time *somebody* is using an accumulator (useful for t=z=1)
def PlotAccumulatorUse(tFactor):
    for t in range(numThreads):
        x = []; y = []
        for r in rows:
            if (r[0] == t) and (r[1] == 2):
                if (r[6] != 0):
                    x.extend([r[5], r[5], r[6], r[6], r[7], r[7], r[3], r[3]])
                    y.extend([0,1,1,0,0,1,1,0])
                else:
                    x.extend([r[5], r[5], r[3], r[3]])
                    y.extend([0,1,1,0])
        x = np.array(x) - t0
        y = np.array(y)
        if t == 0:
            thisLabel = 'accum'
        else:
            thisLabel = None
        # Show the time spent holding any accumulator mutex
        plt.fill_between(x, t*tFactor+0.25, t*tFactor+y/2, color='orange', where=y.astype(np.bool), label=thisLabel)
        
PlotAccumulatorUse(1)
plt.legend()
plt.show()
# Plot a separate figure showing how the accumulator is being used.
# This is really only informative in the 1z,1t case, since I do not currently distinguish between
# the different mutexes, and so the plot just shows whether *any* mutex is held at any given time
plt.figure(figsize=(14,4))
plt.title('Accumulator mutex')
PlotAccumulatorUse(0)
plt.show()

wallclockRunTime = np.max(rows[:,3]) - np.min(rows[:,2])
convolveTime = (np.sum(dt[rows[:,1]==2]-mt[rows[:,1]==2]))
dta = rows[:,3]-rows[:,4]   # Only the accumulator stage of the convolution (including any mutex blocking time)
convolveAccumTime = (np.sum(dta[rows[:,1]==2]-mt[rows[:,1]==2])) # Time spent actively working on the accumulator
print('Wall %.2f, run %.2f, active %.2f, mutex %.2f, idle frac %.2f' % (wallclockRunTime, runTime, runTime-mutexTime, mutexTime, mutexTime/runTime))
print('Parallelism %.1f (Convolve: %.2f, Accum: %.2f)' % ((runTime-mutexTime)/runTime*numThreads, convolveTime/wallclockRunTime, convolveAccumTime/wallclockRunTime))
print('Time breakdown: FH %.2f, mirror %.2f, convolve %.2f, convolve/mutex %.2f' 
          % ((np.sum(dt[rows[:,1]==0])), (np.sum(dt[rows[:,1]==1])),
             convolveTime, (np.sum(mt[rows[:,1]==2]))))


## With one timepoint

### 1 z plane:
Wall 2.44, run 19.32, active 12.59, mutex 6.73, idle frac 0.35
Parallelism 5.2 (Convolve: 1.03, Accum: 0.97)

Time breakdown: FH 9.27, mirror 0.81, convolve 2.51, convolve/mutex 6.73
### 2 z planes:
Wall 3.68, run 28.93, active 26.25, mutex 2.68, idle frac 0.09
Parallelism 7.3 (Convolve: 1.44, Accum: 1.37)
Time breakdown: FH 19.01, mirror 1.92, convolve 5.31, convolve/mutex 2.68

### 4 z planes:
Run 49.41, active 49.11, mutex 0.30, idle frac 0.01
Parallelism 8.0

Time breakdown: FH 34.83, mirror 4.01, convolve 10.27, convolve/mutex 0.30
### Next 4 z planes:
Run 49.40, active 49.13, mutex 0.27, idle frac 0.01
Parallelism 8.0
### 8 z planes:
Run 108.06, active 108.06, mutex 0.00, idle frac 0.00
Parallelism 8.0

## With two timepoints

### 1 z plane:
Wall 2.90, run 23.01, active 15.99, mutex 7.02, idle frac 0.31
Parallelism 5.6 (Convolve: 1.67, Accum: 1.59)
Time breakdown: FH 10.18, mirror 0.96, convolve 4.85, convolve/mutex 7.02

### 2 z planes:
Run 30.14, active 28.15, mutex 2.00, idle frac 0.07
Parallelism 7.5

Time breakdown: FH 15.84, mirror 2.37, convolve 9.94, convolve/mutex 2.00

### 3 z planes:
Run 35.87, active 34.34, mutex 1.53, idle frac 0.04
Parallelism 7.7
### 4 z planes:
Run 64.77, active 64.75, mutex 0.02, idle frac 0.00
Parallelism 8.0 (Convolve: 2.80)

Time breakdown: FH 37.32, mirror 4.55, convolve 22.87, convolve/mutex 0.02

## With 16 timepoints
### 1 z plane
Run 51.78, active 51.78, mutex 0.01, idle frac 0.00
Parallelism 8.0 (Convolve: 6.51)

Time breakdown: FH 8.25, mirror 1.36, convolve 42.16, convolve/mutex 0.01

## With 32 timepoints
### 1 z plane
Run 87.08, active 87.07, mutex 0.01, idle frac 0.00
Parallelism 8.0 (Convolve: 7.11)

Time breakdown: FH 8.36, mirror 1.26, convolve 77.46, convolve/mutex 0.01

## Summary
More time is being lost waiting for a mutex for 1z,2t compared to 2z,1t. This might just be because there is less FH work to be getting on with, and so there are more threads ready to do work.

In fact, with 1z,1t, while threads are inevitably sitting idle I am actually making pretty good use of time - the accumulator mutex is held by somebody almost all the time. However, we are being less efficient with the 2z,t1 and 1z,2t cases. There, we often only hold one of the mutexes at a time. It might be possible to adjust the scheduling code to be more effective with the mutexes (which I think in practice means getting a bit ahead of ourselves with FH rather than waiting until we run out of work entirely). An alternative strategy would be to create additional temporary accumulators and combine them at the end, to support more parallelism. Some perspective though: I do care about the 1z,2t case, but I could only speed it up by 33% (respectable but not earth-shattering); the 4z,2t case is probably more representative of Nils' data, and that has no idle time.